In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import cv2
import dlib

In [ ]:
# Load known faces and encode them
known_faces_dir = "known_faces"
known_faces = []
known_names = []

In [ ]:
for name in os.listdir(known_faces_dir):
    image_path = os.path.join(known_faces_dir, name)
    image = face_recognition.load_image_file(image_path)
    encoding = face_recognition.face_encodings(image)[0]
    known_faces.append(encoding)
    known_names.append(os.path.splitext(name)[0])

In [ ]:
# Initialize variables
face_locations = []
face_encodings = []
face_names = []
attendance = []

In [ ]:
# Load video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_faces, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_faces, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_names[best_match_index]
            if name not in attendance:
                attendance.append(name)

        face_names.append(name)

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.7, (255, 255, 255), 1)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:


# Initialize the face detector and facial landmarks predictor
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Initialize the video capture
video_capture = cv2.VideoCapture(0)

# Variables for blink detection
EYE_AR_THRESH = 0.3  # Eye aspect ratio threshold for blink detection
EYE_AR_CONSEC_FRAMES = 3  # Number of consecutive frames for a valid blink
blink_counter = 0  # Counter for consecutive frames with closed eyes
total_blinks = 0  # Total number of blinks

# Function to calculate the eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    # Compute the Euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])

    # Compute the Euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = distance.euclidean(eye[0], eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # Return the eye aspect ratio
    return ear

while True:
    # Read a frame from the video capture
    ret, frame = video_capture.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_detector(gray)

    # Iterate over the detected faces
    for face in faces:
        # Detect the facial landmarks
        landmarks = landmark_predictor(gray, face)
        landmarks = shape_to_np(landmarks)

        # Extract the left and right eye landmarks
        left_eye = landmarks[l_start:l_end]
        right_eye = landmarks[r_start:r_end]

        # Calculate the eye aspect ratios
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)

        # Average the eye aspect ratios
        ear = (left_ear + right_ear) / 2.0

        # Check if the eye aspect ratio is below the blink threshold
        if ear < EYE_AR_THRESH:
            blink_counter += 1
        else:
            # If the eyes were closed for a sufficient number of frames, increment the total blink count
            if blink_counter >= EYE_AR_CONSEC_FRAMES:
                total_blinks += 1
            # Reset the blink counter
            blink_counter = 0

    # Display the frame with blink count information
    cv2.putText(frame, f"Blinks: {total_blinks}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Liveness Detection", frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video_capture.release()
cv2.destroyAllWindows()


In [ ]:
# Save attendance to a file
with open('attendance.txt', 'w') as file:
    file.write("Attendance:\n")
    for name in attendance:
        file.write(name + "\n")

cap.release()
cv2.destroyAllWindows()